# SlideRule API: Interactive Widget

SlideRule is an on-demand science data processing service that runs in on Amazon Web Services and responds to REST API calls to process and return science results.  SlideRule was designed to enable researchers and other data systems to have low-latency access to custom-generated, high-level, analysis-ready data products using processing parameters supplied at the time of the request. 

The SlideRule ICESat-2 plug-in is a cloud-optimized version of the [ATL06 algorithm](https://nsidc.org/sites/nsidc.org/files/technical-references/ICESat2_ATL06_ATBD_r005.pdf) that can process the lower-level [ATL03 geolocated photon height data products](https://nsidc.org/data/atl03) hosted on AWS by the NSIDC DAAC.  

[Documentation for using SlideRule](http://icesat2sliderule.org/rtd) is available from the [project website](http://icesat2sliderule.org) 

### Background
SlideRule creates a simplified version of the [ICESat-2 ATL06 land ice height product](https://nsidc.org/data/atl06) that can be adjusted to suit different needs.  SlideRule let's you create customized ICESat-2 segment heights _directly_ from the photon height data anywhere on the globe, _on-demand_ and quickly.

### Jupyter and SlideRule
[Jupyter widgets](https://ipywidgets.readthedocs.io) are used to set parameters for the SlideRule API.  

Regions of interest for submitting to SlideRule are drawn on a [ipyleaflet](https://ipyleaflet.readthedocs.io) map.  
The results from SlideRule can be displayed on the interactive [ipyleaflet](https://ipyleaflet.readthedocs.io) map along with additional contextual layers.

#### Load necessary packages

In [ ]:
from sliderule import icesat2, ipysliderule, io
import ipywidgets as widgets
import logging
import warnings
# autoreload
%load_ext autoreload
%autoreload 2
# turn off warnings for demo
warnings.filterwarnings('ignore')

#### Initiate SlideRule API
- Sets the URL for accessing the SlideRule service
- Builds a table of servers available for processing data

In [ ]:
# set the url for the sliderule service
# set the logging level
icesat2.init("icesat2sliderule.org", loglevel=logging.WARNING)

#### Set options for making science data processing requests to SlideRule

SlideRule follows a streamlined version of the [ATL06 land ice height algorithm](https://nsidc.org/sites/nsidc.org/files/technical-references/ICESat2_ATL06_ATBD_r005.pdf).

SlideRule also can use different sources for photon classification before calculating the average segment height.  
This is useful for cases where there may be a vegetated canopy affecting the spread of the photon returns.
- ATL03 photon confidence values, based on algorithm-specific classification types for land, ocean, sea-ice, land-ice, or inland water
- [ATL08 Land and Vegetation Height product](https://nsidc.org/data/atl08) photon classification
- Experimental YAPC (Yet Another Photon Classification) photon-density-based classification

In [ ]:
# display widgets for setting SlideRule parameters
SRwidgets = ipysliderule.widgets()
widgets.VBox([
    SRwidgets.asset,
    SRwidgets.release,
    SRwidgets.classification,
    SRwidgets.surface_type,
    SRwidgets.confidence,
    SRwidgets.quality,
    SRwidgets.land_class,
    SRwidgets.yapc_knn,
    SRwidgets.yapc_win_h,
    SRwidgets.yapc_win_x,
    SRwidgets.yapc_min_ph,
    SRwidgets.yapc_weight,
    SRwidgets.length,
    SRwidgets.step,
    SRwidgets.iteration,
    SRwidgets.spread,
    SRwidgets.count,
    SRwidgets.window,
    SRwidgets.sigma,
])

#### Interactive Mapping with Leaflet

There are 3 projections available within SlideRule for mapping
- [Global (Web Mercator, EPSG:3857)](https://epsg.io/3857)
- [North (Alaska Polar Stereographic, EPSG:5936)](https://epsg.io/5936)
- [South (Antarctic Polar Stereographic, EPSG:3031)](https://epsg.io/3031)

The interactive maps within the SlideRule python API are build upon [ipyleaflet](https://ipyleaflet.readthedocs.io)

#### Leaflet Basemaps and Layers

There are also contextual layers available for each projection.

_[Global (Web Mercator, EPSG:3857)](https://epsg.io/3857)_
- [USGS 3DEP Hillshade](https://apps.nationalmap.gov/3depdem/)
- [ESRI Imagery](https://www.arcgis.com/home/item.html?id=10df2279f9684e4a9f6a7f08febac2a9)
- [Randolph Glacier Inventory (RGI)](http://glims.colorado.edu/glacierdata/)

_[North (Alaska Polar Stereographic, EPSG:5936)](https://epsg.io/5936)_
- [ESRI Imagery](http://goto.arcgisonline.com/maps/Arctic_Imagery)
- [ArcticDEM](https://www.pgc.umn.edu/data/arcticdem)
 
_[South (Antarctic Polar Stereographic, EPSG:3031)](https://epsg.io/3031)_
- [Landsat Image Mosaic of Antarctica (LIMA)](https://lima.usgs.gov/)
- [MODIS Mosaic of Antarctica (MOA)](https://nsidc.org/data/nsidc-0280)
- [Radarsat Antarctic Mapping Project (RAMP)](https://nsidc.org/data/NSIDC-0103)

In addition, most [xyzservice providers](https://xyzservices.readthedocs.io/en/stable/introduction.html) can be added as contextual layers to the global Web Mercator maps

In [ ]:
widgets.VBox([
    SRwidgets.projection,
    SRwidgets.layers
])

#### Select regions of interest for submitting to SlideRule

Here, we create polygons or bounding boxes for our regions of interest.  
This map is also our viewer for inspecting our SlideRule ICESat-2 data returns.

In [ ]:
# create ipyleaflet map in specified projection
m = ipysliderule.leaflet(SRwidgets.projection.value)
# install click handler callback
m.add_selected_callback(SRwidgets.click_handler)
m.map

In [ ]:
m.add_layer(layers=SRwidgets.layers.value)

#### Build and transmit requests to SlideRule

- SlideRule will query the [NASA Common Metadata Repository (CMR)](https://cmr.earthdata.nasa.gov/) for ATL03 data within our region of interest
- When using the `nsidc-s3` asset, the ICESat-2 ATL03 data are then accessed from the NSIDC AWS s3 bucket in `us-west-2`
- The ATL03 granules is spatially subset within SlideRule to our exact region of interest
- SlideRule then uses our specified parameters to calculate average height segments from the ATL03 data in parallel
- The completed data is streamed concurrently back and combined into a geopandas GeoDataFrame within the Python client

In [ ]:
%%time
# sliderule asset and data release
asset = SRwidgets.asset.value
release = SRwidgets.release.value
# build sliderule parameters using latest values from widget
parms = SRwidgets.build()

# create an empty geodataframe
gdf = icesat2.__emptyframe()

# for each region of interest
for poly in m.regions:
    # add polygon from map to sliderule parameters
    parms["poly"] = poly 
    # make the request to the SlideRule (ATL06-SR) endpoint
    # and pass it the request parameters to request ATL06 Data
    gdf = gdf.append(icesat2.atl06p(parms, asset, version=release))

#### Review GeoDataFrame output
Can inspect the columns, number of returns and returns at the top of the GeoDataFrame.

See the [ICESat-2 documentation](http://icesat2sliderule.org/rtd/user_guide/ICESat-2.html#elevations) for descriptions of each column

In [ ]:
print(f'Returned {gdf.shape[0]} records')
gdf.head()

#### Add GeoDataFrame to map

For stability of the leaflet map, SlideRule will as a default limit the plot to have up to 10000 points from the GeoDataFrame

GeoDataFrames can be plotted in any available [matplotlib colormap](https://matplotlib.org/stable/tutorials/colors/colormaps.html)

In [ ]:
widgets.VBox([
    SRwidgets.variable,
    SRwidgets.cmap,
    SRwidgets.reverse,
])

In [ ]:
%matplotlib inline
m.GeoData(gdf, column_name=SRwidgets.variable.value, cmap=SRwidgets.colormap)

#### Remove contextual layers from map

In [ ]:
m.remove_layer(layers=m.layers)

#### Create plots for a single track
Will plot all available cycles of data returned by SlideRule

In [ ]:
widgets.VBox([
    SRwidgets.plot_kind,
    SRwidgets.rgt,
    SRwidgets.ground_track,
    SRwidgets.cycle,
])

In [ ]:
%matplotlib widget
# default is to skip cycles with significant off-pointing
SRwidgets.plot(gdf, kind=SRwidgets.plot_kind.value, cycle_start=3, legend=True)

#### Save GeoDataFrame to output file
- [pytables HDF5](https://www.pytables.org/): easily read back as a Geopandas GeoDataFrame
- [netCDF](https://www.unidata.ucar.edu/software/netcdf): interoperable with other programs

In [ ]:
display(SRwidgets.filesaver)

In [ ]:
# append sliderule api version to attributes
version = icesat2.get_version()
parms['version'] = version['icesat2']['version']
parms['commit'] = version['icesat2']['commit']
# save to file in format (HDF5 or netCDF)
io.to_file(gdf, SRwidgets.file,
    format=SRwidgets.format,
    driver='pytables',
    parameters=parms,
    regions=m.regions,
    verbose=True)

#### Read GeoDataFrame from input file
- [pytables HDF5](https://www.pytables.org/)
- [netCDF](https://www.unidata.ucar.edu/software/netcdf)

In [ ]:
display(SRwidgets.fileloader)

In [ ]:
# read from file in format (HDF5 or netCDF)
gdf = io.from_file(SRwidgets.file,
    format=SRwidgets.format,
    driver='pytables')

#### Review GeoDataFrame input from file

In [ ]:
gdf.head()